# Using Scikit-Learn to train a Binary Classifier.

In this lab, you will use **IBM Watson Studio** to train a model that recognize handwritten digits. This lab is inspired by a popular challenge (and popular dataset - the MNIST) aimed at providing the basics of Machine Learning.  

There is no data assets to load as the data you'll be using is part of **Scikit-Learn**, provided as sample.  

> Scikit-learn is a free software machine learning library for the Python programming language. It features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN, and is designed to interoperate with the Python numerical and scientific libraries NumPy and SciPy.  

**Scikit-Learn** library is part of your **IBM Watson Studio** environment, there is nothing to install. 
If you want to know more about **Scikit-Learn** go here http://scikit-learn.org/

The MNIST dataset is composed of 70,000 images of handwritten digits written by US students and US Federal Agency employees. The dataset is labelled meaning that we know what each image represents. 

Our goal is to train a **Binary Classifier** to recognize digits and get a basic understanding of performance metrics used to score accuracy of models.

## Loading MNIST dataset of handwritten digits
First we'll import the Scikit-Learn library into our environment, and we'll load the MNIST dataset from its sample datasets. 

In [ ]:
import sklearn
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist

Datasets loaded by **Scikit-Learn** have a structure as follow:
- a `DESCR` key which describes the data
- a `DATA` key containing an array. One ligne by observation, one column per feature or variable
- and finally a `target` key as an array of labels.
    
Let's look at that structure in more details.

In [ ]:
X, y = mnist["data"], mnist["target"]
X.shape

In [ ]:
y.shape

There are 70,000 images containing each 784 variables made of the image pixels intensity. The images are 28x28 pixels, which make up 784 values ranging from 0 (white) to 255 (black). This 28x28 array of white/gray/black pixels represent a handwritten digit. 

Next we'll use `MatplotLib` to display a sample of thses 70,000 images converting the 28x28 array of variables into an image.

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
some_digit = X[20000]
some_digit_image = some_digit.reshape(28,28)
plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, 
            interpolation="nearest")
plt.axis("off")
plt.show()

This looks as a **THREE**. Let's retrieve the actual label from the `target` array.

In [ ]:
y[20000]

## Create training and testing datasets
Partitioning datasets into training and testing sets is a **mandatory** step when doing Machine Learning. The MNIST dataset have already been partitioned:
* Training set is made of the 60,000 first images
* Testing set is made of the last 10,000 images

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:],y[:60000],y[60000:]

Machine Learning algorithms may be sensitive to ordered data. The MNIST dataset is ordered, so we will randomly shuffle the observations for better accuracy of the models during the training phase.

In [ ]:
import numpy as np
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

## Train a binary classification model
We'll try to simplify the task of recognizing handwritten digits by first recognizing **only** a single digit, let's say **THREES**. 

In [ ]:
y_train_3 = (y_train == 3)   # True for **threes**, false for the others
y_test_3 = (y_test == 3)

The code above produced two arrays of boolean values from the labelled dataset (the one storing the outcome of a 28x28 pixel array).

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_3)

With our trained model, we can now test it using the `some_digit` image we have instantiated earlier knowing that **it is a THREE**.  
Let's see how our model predicts?

In [ ]:
sgd_clf.predict([some_digit])

The model is good at predicting this actual image (returned value is `True`).  
Next we will measure the performance of the model.

## Performance monitoring
Producing a model is one thing, evaluating it's accuracy is more complex as there are a lot of performance metrics available and different technics.  
We'll start by using a **cross validation** technic to evaluate the model.  

### Cross Validation
**Cross validation** samples the data in different folds using `StratifiedKFold`. Stratification is the process of rearranging the data as to ensure each **fold** is a good representative of the whole. For example in a binary classification problem where each class comprises 50% of the data, it is best to arrange the data such that in every fold, each class comprises around half the instances.  
For each fold:
* we clone our classifier
* train our cloned classifier on the fold
* test the cloned classifier on the testing set
* count the correct predictions and computes a % of accuracy among all the folds.

**Scikit-Learn** has a **cross validation** function as shown below.

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_3, cv=3, scoring="accuracy")

We have a **97%** accuracy rate over all the blocs, surprising and pretty good results.  

**STOP** I need to moderate the entusiasm at this point but before that you can take a look at the below code which implements a **cross validation** without using the function from **Scikit-Learn** for reference.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, random_state=42)

for train_index, test_index in skfolds.split(X_train, y_train_3):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_3[train_index]
    X_test_folds = X_train[test_index]
    y_test_folds = y_train_3[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_folds)
    n_correct = sum(y_pred == y_test_folds)
    print(n_correct / len(y_pred))

Moderate your enthusiasm, let's test a *dummy* classifier which classifies each image outside of the **THREES**.

In [ ]:
from sklearn.base import BaseEstimator
class Never3Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len[X], 1), dtype=bool)

In [ ]:
never_3_clf = Never3Classifier()
cross_val_score(never_3_clf, X_train, y_train_3, cv=3, scoring="accuracy")

The above results show a close to **90%** (even slightly higer) accuracy for classifier that has no intelligence at all. Those *"good"* results are infact due to the fact that predicting non-threes out of a datasets where threes represent 10% of the overall data (MNIST is composed of even observations of 0 to 9) obviously leads to a 90% correctness on average.  

**Bottom line is that accuracy is NOT in general a good performance measure for classifiers, particularly when dealing with asymetric datasets** *(those where certain classes are more frequent than others)*.

### Confusion Matrix
**Confusion matrix** are much more interesting than **accuracy** to evaluate classifier's performance.  
Principle of **confusion matrix** consists of counting the number of times where observations of class A were classified a class B, in our example, how often our classifier has *recognized* threes as fives for instance.  
To build our **confusion matrix** we need a set of predictions to compare to the actual labelled values, put simply "did we predicted correctly?". We'll save our testing dataset for model deployment into production so let's use the function `cross_val_predict()`.

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_3, cv=3)

As for `cross_val_score`, the function `cross_val_predict` does a cross validation in K folds but instead of returning an evaluation score, it returns the **actual predictions** for each fold.  
Now all we need to do is to use the `confusion_matrix` function passing training set and the predicted set as arguments.

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_3, y_train_pred)

The above matrix reads as follow:
* First line represents the non-threes (the negative class). 50,467 images were correctly classified as non-threes, they are **true negatives** (TN)
* Still on the first line, 3,402 were wrongly classified as threes. They are **false positives** (FP)
* Second line of the matrix is all about images representing threes. 675 were wrongly classified, the **false negatives** (FN)
* Whereas remaining 5,456 were correctly classified, we call them the **true positives**  (TP)

The **perfect** classifier would only have true positives and true negatives thus having a confusion matrix with **only** non null values on its main diagonal (upper-left to lower-right). 

In [ ]:
y_train_perfect_predictions=y_train_3
confusion_matrix(y_train_3, y_train_perfect_predictions)

Although **confusion matrix** provides a lot of information a more straigtforward metric would be more interesting. We call a classifier **precision** a metric that evaluates the correctness of positives.   
$$ Precision = TP / ( TP + FP ) $$  

**Precision** is often use along another metric called **recall** or **sensitivity** or also **true positive rate**. It evaluates the positive observations correctness rate.  
$$ Recall = TP / ( TP + FN ) $$  

Hopefully, you don't have to calculate those by yourself, **Scikit-Learn** provides several functions for you. 

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train_3, y_train_pred) 

In [ ]:
recall_score(y_train_3, y_train_pred)

We are now far from the 90% success rate, our classifier isn't so good in the end. The above metrics show that when our classifier tells the image is a three, it is correct 67% of the time and further more it detects only 77% of the threes.  
It is a good habit to combine both metrics in a single one called **score F1**. Score F1 is the weighted average of **precision** and **recall** therefore this score takes both false positives and flase negatives into account. **F1** is more useful than **accuracy** particularly on uneven class distribution.  

There is a good reading here about all those metrics and there usage: [https://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/]  

$$F1 Score = 2 / ( ( 1 / Precision ) + ( 1 / Recall )) = 2 * ( Precision * Recall ) / ( Precision + Recall ) = TP / ( TP + ( FN + FP ) / 2 )$$  

Again, **Scikit-Learn** has a function for this...

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train_3, y_train_pred)

## (Optional) Deploying Scikit-Learn models into IBM Watson Machine Learning

Logical next step once a model is trained is to deploy it. **Scikit-Learn** models can be deployed on **IBM Watson Machine Learning** service using the **Watson Machine Learning Client SDK**.

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials={
    "url": "https://us-south.ml.cloud.ibm.com",
    "username": "5ac9555e-631b-486c-b484-7d6e556c97e3",
    "password": "78c4c71e-5eae-4ccd-926d-d8a40c264d18",
    "instance_id": "76075c55-c628-45c5-9de4-143fbd42238d"
}

client = WatsonMachineLearningAPIClient(wml_credentials)
instance_details = client.service_instance.get_details()
print(instance_details)